In [22]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done


In [23]:
# For connection to Postgres
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-06-24 05:52:46--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.1’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.1s    

2022-06-24 05:52:46 (5.99 MB/s) - ‘postgresql-42.2.9.jar.1’ saved [914037/914037]



In [24]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("amazon_reviews_us_Furniture").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [25]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Luggage_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Luggage_v1_00.tsv.gz"), inferSchema=True, sep="\t")
df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   40884699| R9CO86UUJCAW5|B00VGTN02Y|     786681372|Teenage Mutant Ni...|         Luggage|          3|            0|          0|   N|                Y|my review of this...|my review of this...| 2015-08-31|
|         US|   23208852|R3PR8X6QGVJ8B1|B005KIWL0E|     618251799|Kenneth Cole Reac...|         Luggage|          5|    

In [26]:
# Check data types
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [27]:
# Change review_date to an actual date
from pyspark.sql.types import DateType

df = df.withColumn("review_date", df.review_date.cast(DateType()))
df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   40884699| R9CO86UUJCAW5|B00VGTN02Y|     786681372|Teenage Mutant Ni...|         Luggage|          3|            0|          0|   N|                Y|my review of this...|my review of this...| 2015-08-31|
|         US|   23208852|R3PR8X6QGVJ8B1|B005KIWL0E|     618251799|Kenneth Cole Reac...|         Luggage|          5|    

In [31]:
# Drop duplicates and incomplete rows
print(df.count())
df = df.dropna()
df = df.dropDuplicates()
print(df.count())

348613
348613


In [32]:
# Check data types after changes
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: date (nullable = true)



In [33]:
# Create review_id_table
reviews_df = df.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])
reviews_df.show(5)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R11IBSD5E6HPSD|   20761040|B002B3FWXY|     677901073| 2015-08-31|
|R16F6MZAVR1IYL|   29442013|B000YB7T04|     553070376| 2015-08-31|
|R30889FXLP48F3|   29849475|B00F9KE912|     918753137| 2015-08-30|
|R1DW06GLTNCVWQ|    1988134|B001CDG7Q0|     460451174| 2015-08-29|
| RDC197RTSTZJY|    1302634|B0019GAOO6|      53787504| 2015-08-29|
+--------------+-----------+----------+--------------+-----------+
only showing top 5 rows



In [34]:
# Create products table
products_df = df.select(["product_id", "product_title"])
products_df.show(5)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B002B3FWXY|Travelon Anti-The...|
|B000YB7T04|totes Signature C...|
|B00F9KE912|Samsonite Wheeled...|
|B001CDG7Q0|17 Inch Gray Whee...|
|B0019GAOO6| Everest Hiking Pack|
+----------+--------------------+
only showing top 5 rows



In [35]:
# Count the total number of rows in products_df
counted = products_df.count()
print(f"Total number before: {counted}")

Total number before: 348613


In [40]:
# Only keep distinct products
products_df = products_df.distinct()

# Check the count now
recounted = products_df.count()
print(f"Total number after: {recounted}")

Total number after: 64584


In [41]:

# Create customers table
# Per the schema, create a count of customers
customers_df = df.groupBy("customer_id").count()

# Rename count column
customers_df = customers_df.withColumnRenamed("count", "customer_count")

# Order by customer count
customers_df.orderBy(customers_df["customer_count"].desc()).show(10)

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   12640280|            63|
|   27493011|            31|
|   16600581|            31|
|   43856165|            29|
|   39789300|            26|
|   35461976|            25|
|   35938574|            24|
|   52228204|            23|
|   52955780|            23|
|    5329443|            23|
+-----------+--------------+
only showing top 10 rows



In [42]:
# Check data types for customers_df
customers_df.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_count: long (nullable = false)



In [43]:
from pyspark.sql.types import IntegerType

# Per the schema, convert customer_count to integer
customers_df = customers_df.withColumn("customer_count", customers_df["customer_count"].cast(IntegerType()))

# Check data types after changes
customers_df.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_count: integer (nullable = false)



In [44]:
# Create vine_table
vine_table_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_table_df.show(10)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R11IBSD5E6HPSD|          1|           29|         31|   N|
|R16F6MZAVR1IYL|          4|            0|          0|   N|
|R30889FXLP48F3|          5|            1|          1|   N|
|R1DW06GLTNCVWQ|          5|            0|          0|   N|
| RDC197RTSTZJY|          4|            0|          0|   N|
|R3D44CQ92OA3BR|          5|            0|          0|   N|
|R115NO9WEEUBWD|          4|            1|          1|   N|
|R16QUOK50Y94JU|          5|            0|          0|   N|
| R1P81BC6VL7PZ|          4|            0|          0|   N|
| RZZLHN9MWIAEJ|          5|            0|          0|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 10 rows



In [45]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://<connection>:5432/<db>"
config = {"user":"username", 
          "password": "password", 
          "driver":"org.postgresql.Driver"}

In [46]:
# Write dataframe to review_id_table table in RDS
reviews_df.write.jdbc(url=jdbc_url, table="review_id_table", mode=mode, properties=config)

In [47]:
# Write dataframe to products table in RDS
products_df.write.jdbc(url=jdbc_url, table="products", mode=mode, properties=config)

In [48]:
# Write dataframe to customers table in RDS
customers_df.write.jdbc(url=jdbc_url, table="customers", mode=mode, properties=config)

In [49]:
vine_table_df.write.jdbc(url=jdbc_url, table="vine_table", mode=mode, properties=config)